In [17]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 8.8 MB/s eta 0:00:00:00:010:01m

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [18]:
import spacy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from tqdm import tqdm
from spacy.tokens import DocBin

nlp = spacy.load("ru_core_news_sm")

In [19]:
with open('dataset/toxic_comments.txt', "r") as file:
    file_readen = file.read().split("\n")

labels = np.array(list(map(lambda x: list(map(lambda y: y[9 :], x.split(" ")[0].split(","))), file_readen))[: -1], dtype="object")
text = list(map(lambda x: " ".join(x.split(" ")[1 :]), file_readen))[: -1]

df = pd.DataFrame(np.array([text, labels]).T, columns = ["text", "label"])
print(df.head(15))

                                                 text             label
0                                скотина! что сказать          [INSULT]
1   я сегодня проезжала по рабочей и между домами ...          [NORMAL]
2   очередной лохотрон. зачем придумывать очередно...          [NORMAL]
3   ретро дежавю ... сложно понять чужое сердце , ...          [NORMAL]
4             а когда мы статус агрогородка получили?          [NORMAL]
5   2 августа поздно вечером нашли вот такую потер...          [NORMAL]
6         вчера надыбала новые стикеры #u2a94ec7fabs#          [NORMAL]
7   заколоть этого плешивого урода что бы крякнул ...  [INSULT, THREAT]
8   а еще на стоянке никто не проверяет безопаснос...          [NORMAL]
9   красота..!! если есть, что показать??!! почему...          [NORMAL]
10                                ходунки какая цена?          [NORMAL]
11                      любовь....с первого взгляда..          [NORMAL]
12                                ои уже и етому учат          [

In [20]:
df = df.sample(frac=0.5)
print(df)

                                                     text     label
138489                            мало ли что напридумают  [NORMAL]
38853   больше никто не требуется . объявлению уже 4 г...  [NORMAL]
163377                        такие глаза, ох красавец!!!  [NORMAL]
151783                   царанг ту кати тар связь нахтиям  [NORMAL]
79986                 вот это мразь, чему радуется козёл.  [INSULT]
...                                                   ...       ...
117592  ты чудо чудное. ты милый, красивый, дай тебе б...  [NORMAL]
213107  пидарок какой-то, ну в смысле, просветлённый, ...  [INSULT]
65769                         твои муж дурак из дураков .  [INSULT]
86175   владимир кутузов, включите мозги если они у ва...  [NORMAL]
71164   у меня таксеныш пока рос сгрыз все и диван нор...  [NORMAL]

[124145 rows x 2 columns]


In [21]:
data = [tuple(df.iloc[i].values) for i in range(df.shape[0])]
print(data[:10])

[('мало ли что напридумают', ['NORMAL']), ('больше никто не требуется . объявлению уже 4 года!', ['NORMAL']), ('такие глаза, ох красавец!!!', ['NORMAL']), ('царанг ту кати тар связь нахтиям', ['NORMAL']), ('вот это мразь, чему радуется козёл.', ['INSULT']), ('это апокриф..уже всем известно..так как ранние христиане как и первые о такой библии от варнавы и не слышали и не знали и не читали:-)', ['NORMAL']), ('с днем рождения малыш, здоровья счастья тебе желаю', ['NORMAL']), ('а, что граница открылась?', ['NORMAL']), ('е иbою мать это ж пиздец ееsихпет', ['INSULT']), ('вы тот кем сами себя считаете и видите как правило то, что хотите увидеть. а кто думает, что получая или воруя денег чуть больше чем остальные он свободен, дурак или подлец.', ['INSULT'])]


In [22]:
train_data, valid_data = train_test_split(data, test_size=0.3, random_state=42, shuffle=True)
valid_data, test_data = train_test_split(valid_data, test_size=0.1, random_state=13, shuffle=True)

In [23]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) is way faster than running
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple,
    # the first one is treated as text
    # the second one will get returned as it is.
    # a = tqdm(nlp.pipe(data, as_tuples=True), total = len(data))
    for doc, labels in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        doc.cats = {"normal": 0,
                    "insult": 0,
                    "threat": 0,
                    "obscenity": 0}
        if 'NORMAL' in labels:
          doc.cats["normal"] = 1
        if 'INSULT' in labels:
          doc.cats["insult"] = 1
        if 'THREAT' in labels:
          doc.cats["threat"] = 1
        if 'OBSCENITY' in labels:
          doc.cats["obscenity"] = 1
        # we need to set the (text)cat(egory) for each document
        #doc.cats["positive"] = label
        # put them into a nice list
        docs.append(doc)
    return docs

In [24]:
# we are so far only interested in the first 5000 reviews
# this will keep the training time short.
# In practice take as much data as you can get.
# you can always reduce it to make the script even faster.
num_texts = 5000
# first we need to transform all the training data
train_docs = make_docs(train_data[:num_texts])

100%|██████████| 5000/5000 [00:41<00:00, 120.42it/s]


In [25]:
# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("spacy/train.spacy")
# repeat for validation data
valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("spacy/valid.spacy")

100%|██████████| 5000/5000 [00:38<00:00, 131.36it/s]


на этом месте мы идем в https://spacy.io/usage/training#quickstart, там настраиваем под себя конфиг (textcat), копируем его руками(!) в base_config.cfg, указываем правильные пути до трейн и вэлид


In [26]:
! touch base_config.cfg

In [27]:
! python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [28]:
! python -m spacy train config.cfg --output ./output --paths.train ./spacy/train.spacy --paths.dev ./spacy/valid.spacy

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       51.76    0.52
  0     200          32.30       55.48    0.55
  0     400          21.63       57.02    0.57
  0     600          20.61       59.31    0.59
  1     800          17.37       62.46    0.62
  2    1000          16.00       65.84    0.66
  2    1200          13.70       69.38    0.69
⚠ Aborting and saving the final best model. Encountered exception:
FileNotFoundError(2, 'No such file or directory')
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/

In [29]:
import spacy
# load the best model from training
nlp = spacy.load("output/model-best")    
for text, labels in test_data[:20]:
    doc = nlp(text)
    print(f"Sample: {text}\nLabels: {labels}\nOutput: {doc.cats}\n")

Sample: ребёнок голоден!
Labels: ['NORMAL']
Output: {'normal': 0.6493539810180664, 'insult': 0.33670878410339355, 'threat': 0.2311621457338333, 'obscenity': 0.18510280549526215}

Sample: и правда! идёшь выносить мусор- а баков нет. зато по пути к бывшей стоянке дорожка из пакетов с мусором. кто мудрит?!
Labels: ['NORMAL']
Output: {'normal': 0.9294244647026062, 'insult': 0.042160872370004654, 'threat': 0.0008697120938450098, 'obscenity': 0.00016168357979040593}

Sample: хм..., придумают же! 👏
Labels: ['NORMAL']
Output: {'normal': 0.760574221611023, 'insult': 0.22027620673179626, 'threat': 0.07510235905647278, 'obscenity': 0.04189739748835564}

Sample: дать пожизненный срок и каждый день приходить вкамеру
Labels: ['NORMAL']
Output: {'normal': 0.6885133981704712, 'insult': 0.3033641278743744, 'threat': 0.1819431036710739, 'obscenity': 0.1239987388253212}

Sample: главное,чтобы она жила
Labels: ['NORMAL']
Output: {'normal': 0.6424776911735535, 'insult': 0.32481297850608826, 'threat': 0.175